In [1]:
from seshat_api import SeshatAPI

In [ ]:
# Set up a client
client = SeshatAPI(username="<USERNAME>", password="<PASSWORD>")

In [ ]:
# Get all the polities in the database
from seshat_api.core import Polities

polities = Polities(client)

# Show me how many polities are in the database
print(len(polities))

In [ ]:
# Iterate through the polities
for polity in polities:
    print(polity.id, polity.name)